## Resume Word Analysis
This notebook analyzes the frequency of words in a resume and visualizes the top terms before and after removing stop words.

In [ ]:
import re
from collections import Counter
from pathlib import Path

resume_path = Path('Q2/data/suprawee_resume.txt')
stop_words_path = Path('Q2/data/stop_words.txt')
figures_dir = Path('Q2/figures')
figures_dir.mkdir(parents=True, exist_ok=True)

def save_svg_bar(data, filename, title):
    width = 800
    height = 400
    bar_width = width / len(data)
    max_count = max(count for _, count in data)
    scale = (height - 50) / max_count
    svg_parts = [f'<svg xmlns="http://www.w3.org/2000/svg" width="{width}" height="{height}">']
    svg_parts.append(f'<text x="{width/2}" y="20" text-anchor="middle" font-size="16" fill="white">{title}</text>')
    for i, (word, count) in enumerate(data):
        x = i * bar_width
        bar_height = count * scale
        y = height - bar_height - 20
        svg_parts.append(f'<rect x="{x}" y="{y}" width="{bar_width-2}" height="{bar_height}" fill="steelblue"/>')
        svg_parts.append(f'<text x="{x + bar_width/2}" y="{height - 5}" text-anchor="middle" font-size="10" transform="rotate(45 {x + bar_width/2},{height - 5})" fill="white">{word}</text>')
    svg_parts.append('</svg>')
    Path(filename).write_text('
'.join(svg_parts), encoding='utf-8')

text = resume_path.read_text(encoding='utf-8')
words = re.findall(r'\b\w+\b', text.lower())
word_counts = Counter(words)
top_resume = word_counts.most_common(20)
save_svg_bar(top_resume, figures_dir/'resume_words.svg', 'Top 20 Resume Words')
top_resume[:5]


In [ ]:
stop_words = set(stop_words_path.read_text(encoding='utf-8').split())
specific_words = [w for w in words if w not in stop_words]
specific_counts = Counter(specific_words)
top_specific = specific_counts.most_common(20)
save_svg_bar(top_specific, figures_dir/'specific_words.svg', 'Top 20 Specific Words')
top_specific[:5]


Removing common stop words reveals keywords that emphasize technical skills and experiences, such as **developed**, **python**, and **project**, giving more insight into the resume's content.

## Class Resume Word Analysis
Analyze word usage across all resumes in the class.


In [ ]:
from PyPDF2 import PdfReader
resumes_dir = Path('Q2/data/resumes')
all_text = []
for pdf_path in sorted(resumes_dir.glob('*.pdf')):
    try:
        reader = PdfReader(str(pdf_path))
        text = ''
        for page in reader.pages:
            text += page.extract_text() or ''
        all_text.append(text)
    except Exception as e:
        print(f'Failed to read {pdf_path}: {e}')
class_text = '
'.join(all_text)
Path('Q2/data/resumes_text.txt').write_text(class_text, encoding='utf-8')
class_words = re.findall(r'\b\w+\b', class_text.lower())
class_counts = Counter(class_words)
top_class = class_counts.most_common(20)
save_svg_bar(top_class, figures_dir/'class_resume_words.svg', 'Top 20 Resume Words (Class)')
top_class[:5]


In [ ]:
class_specific_words = [w for w in class_words if w not in stop_words]
class_specific_counts = Counter(class_specific_words)
top_class_specific = class_specific_counts.most_common(20)
save_svg_bar(top_class_specific, figures_dir/'class_specific_words.svg', 'Top 20 Specific Words (Class)')
top_class_specific[:5]


Removing stop words across all resumes highlights technical and domain-specific terms that reflect common skills among classmates.


In [ ]:
my_specific_set = set(specific_words)
class_specific_set = set(class_specific_words)
unique_words = sorted(my_specific_set - class_specific_set)[:20]
unique_words


These words appear only in my resume, distinguishing my experiences from those of classmates.
